In [5]:
# Esta es una modificacion del script original para google colab, para que funcione en github codespaces
# Hecho por @LUCAST o @Lucast1574 o @Lucast_1574 en github y otras redes sociales
# Versiones disponibles testeadas:
# - 1.20.4
# - 1.20.3
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.4
# - 1.19.3
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.5
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8


version = '1.20.1' # Version de Minecraft a usar

#Chose server type. Currently available versions:
# - paper (Lo más vanilla. Es lo mismo que SPIGOT, bukkit, etc)
# - forge (Para usar MODS. Es básicamente FORGE) [Aviso, la instalación tarda APROX 10-15 min, al ejecutar el primer paso, esperar a que ponga COMPLETADO]
# - fabric (Para jugar Fabric. Es una alternativa a Forge)
# - vanilla (Minecraft Vanilla sin mods ni plugins)
# - snapshot (En "version" tendrás que poner el nombre de la SNAPSHOT, EJ: 23w42a)
# - mohist (Versión de Minecraft que combina Forge y Bukkit)

server_type = 'mohist'

#Importa librerias necesarias para el funcionamiento del script

import os
import requests
import json



# Guarda el archivo de configuración localmente
colabconfig = {"server_type": server_type, "server_version": version}
with open("colabconfig.json", 'w') as config_file:
    json.dump(colabconfig, config_file)

# Imprime un mensaje de finalización
print('Completado!')
# Cambie el eula a true para aceptar los términos de Minecraft
!echo "eula=true" >> eula.txt


Completado!


In [1]:
#importa librerias necesarias para ejecucion

import requests

import time

import os

import re

import json

import glob

import threading



#Actualizacion de lista de paquetes

!sudo apt update &>/dev/null && echo "el apt cache fue actualizado correctamente" || echo "la actualizacion del apt cache fallo, es posible que recibas paquetes obsoletos"

#Asignar archivos al directorio


os.makedirs("/workspaces/codespaces-jupyter/Minecraft-server", exist_ok=True)

os.chdir("/workspaces/codespaces-jupyter/Minecraft-server")

!ls #lista los archivos del directorio para verificarlos



#Importacion del archivo de configuracion


config_path = "/workspaces/codespaces-jupyter/Minecraft-server/colabconfig.json"

if os.path.isfile(config_path):

    with open(config_path) as config_file:

        colabconfig = json.load(config_file)

else:

    colabconfig = {"server_type": "generic"} # Usar la configura default si no existe

    with open(config_path, 'w') as new_config_file:

        json.dump(colabconfig, new_config_file)



# Instalar OpenJDK


version = colabconfig["server_version"]

if colabconfig["server_version"] < "1.17":

    !sudo apt-get purge openjdk* > /dev/null 2>&1

    !sudo apt-get install openjdk-8-jre-headless &>/dev/null and print("OpenJDK 8 instalado!") or print("Error al intalar OpenJDK 8.")

else:

    !sudo apt-get purge openjdk* > /dev/null 2>&1

    !sudo apt-get install openjdk-17-jre-headless &>/dev/null and print("OpenJDK 17 instalado!") or print("Error al intalar OpenJDK 17.")


# Chequeo de version de java

java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

if java_ver[0] == "17":

    print("Estas usando JAVA 17.")

else:

    print("Estas usando una version desactualizada de java. Puedo que Minecraft 1.17+ no funcione.")



#Lista de posibles archivos .jar


jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar', 'mohist': "mohist.jar"}

jar_name = jar_list[colabconfig["server_type"]]



# Java arguments.

if colabconfig["server_type"] == "paper":

    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"

else:

    server_flags = "" # No se necesitan banderas para servidores que no sean de paper

#Argumentos JVM

memory_allocation = "-Xms10G -Xmx10G"



# Elige el servicio de tunelacion que desees

# Opciones disponibles: ngrok

tunnel_service = "ngrok"

print("Usando", tunnel_service)


#Condicional de tunelacion del servidor



if tunnel_service == "ngrok":

    !pip -q install pyngrok #Aqui se instala el ngrok en el servidor

    from pyngrok import conf, ngrok



    # Obten tu token de ngrok

    print("Consigue tu authtoken de https://dashboard.ngrok.com/auth") #Consigue aqui tu token de ngrok, sera dinamico, si quieres que sea estatico, necesitaras la version premiun de ngrok

    import getpass



    # v - - - - - - - TOKEN - - - - - - - v



    authtoken = "2d1iTcQmdKTStxglJnv0hdCxTcw_6TaMwZLz5aCxKFYCnQgb" # <---- TOKEN NGROK (PON AQUÍ TU TOKEN [TIENE QUE ESTAR ENTRE LAS "COMILLAS"])

    ! ngrok authtoken $authtoken # logeo en ngrok



    #Establecimiento de region de ngrok



    #v - - - - - - - REGIONES DISPONIBLES - - - - - - - v

    # ap - Asia/Pacifico (Singapore)

    # au - Australia (Sydney)

    # eu - Europa (Frankfurt - Alemania)

    # in - India (Mumbai)

    # jp - Japon (Tokyo)

    # sa - America del sur (São Paulo - Brasil)

    # us - Estados unidos (Ohio)

    conf.get_default().region = 'us' # <--- Cambia esto por la region que quieras



    # Conectar a ngrok

    url = ngrok.connect(25565, 'tcp')

    print('La IP de tu servidor es ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

    #Si obtienes el servicio premiun de ngrok borra las dos lineas de codigo de arriba y quitale el # a estas a continuacion:

    #subdominio = "mi-subdominio"

    #url = ngrok.connect(25565, 'tcp', subdomain=subdominio)

    #print('La IP de tu servidor es ' + url.replace('tcp://', ''))



print('Iniciando servidor...')

def get_users():
    url = 'https://jsonplaceholder.typicode.com/users'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print("Lista de usuarios:")
            for user in data:
                print(f"Nombre: {user['name']}, Email: {user['email']}")
        else:
            print(f"Error al hacer la solicitud. Código de estado: {response.status_code}")
    except requests.RequestException as e:
        print(f"Error de conexión: {e}")

# Función para ejecutar get_users() cada 25 minutos
def repeat_get_users():
    while True:
        get_users()
        time.sleep(600)  # Espera 10 minutos

# Inicia un hilo para ejecutar repeat_get_users() en segundo plano
user_thread = threading.Thread(target=repeat_get_users)
user_thread.start()

if colabconfig["server_type"] == "forge":

    version = colabconfig["server_version"]

    if colabconfig["server_version"] < "1.17":

        oldpathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/forge-{version}-*.jar")

        if oldpathtoforge: # Checa si la lista no esta vacia

            path = oldpathtoforge[0] # Obtiene la primera ruta de la lista

            print(path)

            !java $memory_allocation -jar "{path}" nogui

        else:

            print("No se encontro la version de Forge universal.")

    else:

        pathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/1.20.1-47.2.0/unix_args.txt")

        if pathtoforge: # Checa si la lista no esta vacia

            path = pathtoforge[0] # Obtiene la primera ruta de la lista

            print(path)

            !java @user_jvm_args.txt "@{path}" "$@"

        else:

            print("No unix_args.txt found.")
else:

    !java $memory_allocation $server_flags -jar $jar_name nogui



# Ciclo para realizar las solicitudes a JSONPlaceholder cada minuto
 
while True:
    time.sleep(60)

el apt cache fue actualizado correctamente
banned-ips.json      eula.txt	    mods	     run.sh		world
banned-players.json  forge.jar	    mohist.jar	     server.properties
colabconfig.json     forge.jar.log  ops.json	     user_jvm_args.txt
config		     libraries	    patchouli_books  usercache.json
defaultconfigs	     logs	    run.bat	     whitelist.json
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `sudo apt-get install openjdk-17-jre-headless &>/dev/null and print("OpenJDK 17 instalado!") or print("Error al intalar OpenJDK 17.")'
Estas usando una version desactualizada de java. Puedo que Minecraft 1.17+ no funcione.
Usando ngrok
Consigue tu authtoken de https://dashboard.ngrok.com/auth
Authtoken saved to configuration file: /home/codespace/.config/ngrok/ngrok.yml
La IP de tu servidor es 0.tcp.ngrok.io:13968
Iniciando servidor...
/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/1.20.1-47.2.0/unix_args.txt
Lista de u